### Intents.json – The data file which has predefined patterns and responses.
### train_chatbot.py – A script to build the model and train our chatbot.
### Words.pkl – Contains a list of our vocabulary.
### Classes.pkl – Contains the list of categories.
### Chatbot_model.h5 – Trained model (with model info like weights of the neurons).
### Chatgui.py – Script implementing GUI for our chatbot. Users can easily interact with the bot.


## Import and load the data file

The data file is in JSON format so we used the json package to parse the JSON file into Python.

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('punkt') #needed for tokenizing classes (preprocessing)
nltk.download('wordnet') #needed for lemmatizing
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)
print(intents) #nice

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], 'responses': ['I can guide you through

2021-10-24 19:55:49.766105: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-24 19:55:49.766128: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Preprocess data

Tokenizing text data (basic first step) also create a list of classes for our tags.

In [2]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

### lemmatize each word and remove duplicate words from the list
#### lemmatize = sort words by grouping inflected or variant forms of the same word. nltk handles it

In [3]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


## Create training and testing data

    Create the training data, we will provide the input and the output.
    Input will be the pattern
    Output will be the class that input pattern belongs to
    Computer doesn’t understand text so we will convert text into numbers.

In [4]:
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object) #we are using tuples in the array so use object bc immutableness
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


## Build the model

    We have our training data ready
    now we will build a deep neural network that has 3 layers. 
    use the Keras sequential API 

In [5]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200


2021-10-24 19:55:52.012597: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-24 19:55:52.012630: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-24 19:55:52.012649: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (CS): /proc/driver/nvidia/version does not exist
2021-10-24 19:55:52.012830: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/ubuntu/anaconda3/envs/ML/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr`

10/10 [==============================] - 0s 1ms/step - loss: 2.2557 - accuracy: 0.1277
Epoch 2/200
10/10 [==============================] - 0s 1ms/step - loss: 2.1372 - accuracy: 0.2128
Epoch 3/200
10/10 [==============================] - 0s 1ms/step - loss: 2.0151 - accuracy: 0.2979
Epoch 4/200
10/10 [==============================] - 0s 1ms/step - loss: 1.9663 - accuracy: 0.3404
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 1.8499 - accuracy: 0.4681
Epoch 6/200
10/10 [==============================] - 0s 1ms/step - loss: 1.7396 - accuracy: 0.4255
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 1.5368 - accuracy: 0.6170
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.4763 - accuracy: 0.5957
Epoch 9/200
10/10 [==============================] - 0s 1ms/step - loss: 1.3762 - accuracy: 0.5745
Epoch 10/200
10/10 [==============================] - 0s 1ms/step - loss: 1.1418 - accuracy: 0.7021
Epoch 11/200
10/10 [=

10/10 [==============================] - 0s 1ms/step - loss: 0.0277 - accuracy: 1.0000
Epoch 84/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0826 - accuracy: 1.0000
Epoch 85/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0712 - accuracy: 1.0000
Epoch 86/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0522 - accuracy: 0.9787
Epoch 87/200
10/10 [==============================] - 0s 960us/step - loss: 0.0327 - accuracy: 1.0000
Epoch 88/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0362 - accuracy: 0.9787
Epoch 89/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0129 - accuracy: 1.0000
Epoch 90/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0563 - accuracy: 1.0000
Epoch 91/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0227 - accuracy: 1.0000
Epoch 92/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0252 - accuracy: 1.0000
Epoch 93/20

### 200 epochs later, the accuracy regularly tests at 100% (on training data)


## Predict the response 

    We will load the trained model, then use a GUI to predict the response from the bot. 
    model only tells us the class it belongs to
    so we implement some functions to identify the class, then retrieve a random response from the list of responses.

    import the necessary packages and load the ‘words.pkl’ and ‘classes.pkl’ pickle files which we have created when we trained our model:

In [6]:
#already made as .py's
%run chatApp.py
%run chatGUI.py